<a href="https://colab.research.google.com/github/takaito1/DOMIP/blob/main/bin_wod_bottle_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bin bottle data from WOD
  - prepared by T. Ito, 2024
  - reads in pre-downloaded WOD T/S/O2 data (2022-02)
  - binning the data into cells at a given resolution
  - saves the results as a netCDF file

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/drive') # this links your google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dosd='/content/drive/MyDrive/DOMIP/WOD18_Feb22/ocldb1646245120.16765_OSD'
Nosd=37 # number of OSD files

In [4]:
# prepare grid cells
res=1.0
xW=np.arange(-180,180,res)        # western edge of the cell
yS=np.arange(-90,90,res)          # southern edge of the cell
xC=np.arange(-180,180,res)+res/2  # x-center of the cell
yC=np.arange(-90,90,res)+res/2    # y-center of the cell
xE=np.arange(-180,180,res)+res    # eastern edge of the cell
yN=np.arange(-90,90,res)+res      # northern edge of the cell
# (Nx,Ny) are the size of the x and y cells
Nx = np.size(xC)
Ny = np.size(yC)

In [5]:
# standard depth above 6km
ds   = xr.open_dataset(f'{dosd}.nc')
zall = ds.z.to_numpy()
Z    = np.unique(zall[zall<=6000.])  # cell center depth
Nlev = np.size(Z)

In [6]:
# define a function to extracts WOD bottle profile
# from a specific year and month this Nlev vertical levels
# and with WOD flag = 0 (accepteable data) for the named variable
# (Temperature, Salinity or Oxygen)
#
def monthly_1deg_bin(Nlev,year,month,varname):
    # prepare array
    dd_osd=np.zeros((Nlev,Ny,Nx),dtype='int')
    sumX_osd=np.zeros((Nlev,Ny,Nx))
    dd_prof=np.zeros((Ny,Nx),dtype='int')
    # loop over each WOD ragged netCDF file
    for f in range(Nosd):
        if f==0:
            fn=dosd+'.nc'
        else:
            fn=dosd+str(f+1)+'.nc'
        #print('processing '+fn)
        dsosd=xr.open_dataset(fn)
        # now loop over each cast in the netCDF file
        timeloc = pd.DatetimeIndex(dsosd['time'])
        Nprof0=np.size(timeloc)
        yearloc = timeloc.year
        uyear=np.unique(yearloc)
        if uyear[0]==1770:
            minyr=uyear[1]
        else:
            minyr=uyear[0]
        # get month
        monloc = timeloc.month
        maxyr=uyear[-1]
        if (maxyr < year) | (minyr > year):
            note='skipping non-overlapping years...'
        else:
            print('processing '+fn)
            print('num of prof = '+str(Nprof0)+' max year = '+str(maxyr)+' min year = '+str(minyr))
            print('reading data...')
            lonloc=dsosd['lon'].to_numpy()
            latloc=dsosd['lat'].to_numpy()
            # get row size (z,T,S) are the same
            zrsize=dsosd['z_row_size'].to_numpy()
            orsize=dsosd[varname+'_row_size'].to_numpy()
            # get depth data
            zp0=dsosd['z'].to_numpy()
            # get tracer data
            o2p0=dsosd[varname].to_numpy()
            oflg0=dsosd[varname+'_WODflag'].to_numpy()
            zlevidx=0
            o2levidx=0
            lonloc[lonloc==-180.0]=-179.99
            for nn in range(Nprof0):
                # current year and month
                monow=monloc[nn]
                yrnow=yearloc[nn]
                # determine x,y,month grid
                if (lonloc[nn]<180.0)&(lonloc[nn]>-180)&(latloc[nn]<90.0)&(latloc[nn]>-90):
                    xind=np.where((xW>=lonloc[nn]-res)&(xW<lonloc[nn]))[0][0]
                    yind=np.where((yS>=latloc[nn]-res)&(yS<latloc[nn]))[0][0]
                    goodxy=1
                else:
                    goodxy=0
                # determine the number of samples in this particular profile
                o2inc=orsize[nn]
                # obtain depth of the samples
                zp1=zp0[zlevidx:int(zlevidx+zrsize[nn])]
                # check if O2 data exists
                if np.isnan(o2inc)|(goodxy==0):
                    tmp_note='no data'
                else:
                    # get the profile and its quality flag
                    Nz1=int(o2inc)
                    o2p1=o2p0[o2levidx:int(o2levidx+o2inc)]
                    oflg1=oflg0[o2levidx:int(o2levidx+o2inc)]
                    o2levidx=int(o2levidx+o2inc)
                    if (Nz1>Nlev):
                        Nz1=Nlev
                    for k in range(Nz1):
                        # Use data with WODflag = 0 (accepted data)
                        if (oflg1[k]==0)&(yrnow==year)&(monow==month):
                            sumX_osd[k,yind,xind]=sumX_osd[k,yind,xind]+o2p1[k]
                            dd_osd[k,yind,xind]=dd_osd[k,yind,xind]+1
                            if (k==0):
                                #print(monow)
                                dd_prof[yind,xind]=dd_prof[yind,xind]+1
                    # --- done within profile loop
                # -------
                # now update the index for Z, T and S and go to the next profile
                zlevidx=int(zlevidx+zrsize[nn])
            # check if the number of sample matches up for the entire WOD file
            Nobs0=np.size(zp0)
            if zlevidx!=Nobs0:
                print('error! number of sample did not match')
            else:
                print('passed the sample count check')
    return sumX_osd,dd_osd,dd_prof

In [7]:
# now bin the data one month at a time
start_year = 1965
end_year = 2020
variable_name='Oxygen'
yrs = np.arange(start_year,end_year+1,1)
Nyrs=np.size(yrs)
#
data=np.zeros((Nyrs*12,Nlev,Ny,Nx))
count=np.zeros((Nyrs*12,Nlev,Ny,Nx))
cnt=0
#
# assemble binned O2 data
for n,year in enumerate(yrs):
    for mn in range(12):
        print(f'working on {year}-{mn+1} ...')
        sumX_osd,dd_osd,dd_prof = monthly_1deg_bin(Nlev,year,mn+1,variable_name)
        data[cnt,:,:,:] = sumX_osd/dd_osd
        count[cnt,:,:,:]= dd_osd
#


working on 1965-1 ...
processing /content/drive/MyDrive/DOMIP/WOD18_Feb22/ocldb1646245120.16765_OSD9.nc
num of prof = 69441 max year = 1965 min year = 1963
reading data...
passed the sample count check
processing /content/drive/MyDrive/DOMIP/WOD18_Feb22/ocldb1646245120.16765_OSD10.nc
num of prof = 67692 max year = 1966 min year = 1965
reading data...


KeyboardInterrupt: 

In [ ]:
# save as a netCDF file
time=np.arange(f'{start_year}-01',f'{end_year+1}-01',dtype='datetime64[M]')
da=xr.DataArray(data=data,name=variable_name,dims=['time','depth','lat','lon'],
                coords={'time':time,'depth':Z,'lat':yC,'lon':xC})
ds=da.to_dataset()
ds['count']=xr.DataArray(data=count,name='count',dims=['time','depth','lat','lon'],
                coords={'time':time,'depth':Z,'lat':yC,'lon':xC})
ds.to_netcdf(f'{variable_name}_WOD_QC0_{start_year}_{end_year}.nc')